In [0]:
from pyspark.sql.functions import *
import json

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS DEV;
CREATE SCHEMA IF NOT EXISTS DEV.bronze;
CREATE SCHEMA IF NOT EXISTS DEV.silver;
CREATE SCHEMA IF NOT EXISTS DEV.gold;

In [0]:
%sql
USE CATALOG DEV;
USE SCHEMA bronze;

In [0]:
with open(
    "/Volumes/dev/bronze/volumes/config/metadata_config.json",
    "r"
) as f:
    config = json.load(f)

In [0]:
for key, value in config['datasets'].items():
    if value['file_type']=='csv':
        df=spark.read.csv(path=value['file_path'],header=True,inferSchema=True)
        df.write.mode("overwrite").saveAsTable(value['target_table_name'])
    elif value['file_type']=='json':
        df=spark.read.json(path=value['file_path'],multiLine=True)
        if value['file_name']=='accounts.json':
            df = df.select(
                    "*",
                    "nominee_details.nominee_name",
                    "nominee_details.relationship"
                ).drop("nominee_details")
        elif value['file_name']=='branches.json':
            df =  df.select(
                    "*",
                    "address.street_address",
                    "address.city",
                    "address.state",
                    "address.pincode",
                    "address.landmark",
                    "contact_details.email",
                    "contact_details.fax",
                    "contact_details.phone_numbers"
            ).drop("address","contact_details")
        else:
            pass
        df=df.withColumn("ingestion_date",current_timestamp()).withColumn("path",col("_metadata.file_path"))
        df.write.mode("overwrite").saveAsTable(value['target_table_name'])